### Temat 1. Własna sieć neuronowa
Przykładowe zadanie: klasyfikacja kwiatów irysa (Fisher's iris dataset)

In [ ]:
using Pkg
Pkg.activate("./project")
Pkg.add("MLDatasets")
Pkg.add("IJulia")
Pkg.add("StableRNGs")
Pkg.add("DataFrames")
Pkg.add("PyPlot")
Pkg.add("MLDataUtils")

In [ ]:
using RDatasets
using Random
using Plots
using DataFrames
using MLDataUtils
using Random
using Statistics

Wczytanie danych

In [ ]:
iris = Matrix(dataset("datasets", "iris"))
using Random


In [ ]:
X = iris[:,1:4]
y = iris[:,5]
println("X data shape: ", size(X))
println("Y data shape: ", size(y))

In [ ]:
species = ["setosa", "virginica", "versicolor"]

In [ ]:
function reshape_data(data_x, data_y,species_vec)
    x_reshaped = []
    y_reshaped = []
    
    for i = 1:size(data_x)[1]
        push!(x_reshaped, data_x[i,:])
        y = zeros(3)
        if data_y[i] == species_vec[1]
            y[1] = 1.0
        elseif data_y[i] == species_vec[2]
            y[2] = 1.0
        elseif data_y[i] == species_vec[3]
            y[3] = 1.0
        end
        push!(y_reshaped,y)
    end
    
    d = [data for data in zip(x_reshaped, y_reshaped)]
    
    return d
end

data = reshape_data(X, y,species)

Podział na dane treningowe i testowe

In [ ]:
Random.seed!(1234)
train, test = splitobs(shuffleobs(data), at = 0.8)

Różniczkowanie w przód

In [ ]:
J = function jacobian(f, args::Vector{T}) where {T <:Number}
    jacobian_columns = Matrix{T}[]
    
    for i=1:length(args)
        x = Dual{T}[]
        for j=1:length(args)
            seed = (i == j)
            push!(x, seed ?
                Dual(args[j], one(args[j])) :
                Dual(args[j],zero(args[j])) )
        end
        column = partials.([f(x)...])
        push!(jacobian_columns, column[:,:])
    end
    hcat(jacobian_columns...)
end

In [ ]:
mutable struct Layer
    m::Int
    n::Int
    activation::Function
    W::Matrix
    dW::Matrix
    b::Vector
    db::Vector
end


In [ ]:
function NeuralNetwork()
    layer = []
    function AddLayer(m,n,activation) layer_::Layer = Layer(m,n,activation,randn(n,m),randn(n,m),randn(n),randn(n)); push!(layer, layer_);end
    () -> (AddLayer,layer)
end

In [ ]:
forward(net, x, y) =
let
    for i=1:(size(net.layer)[1])
        x = net.layer[i].activation.(reshape(net.layer[i].W, net.layer[i].n, net.layer[i].m) * x .+ net.layer[i].b)
        end 
        E = mean_squared_loss(y, x)
        return E
    end

In [ ]:
forward_w(net, x, y, w, j) =
    let 
        tmp = randn(net.layer[j].n, net.layer[j].m)
        tmp[:,:] = net.layer[j].W
        net.layer[j].W = w[:, :]
    
        for i=1:(size(net.layer)[1])
            x = net.layer[i].activation.(reshape(net.layer[i].W, net.layer[i].n, net.layer[i].m) * x .+ net.layer[i].b)
        end
         
        net.layer[j].W = tmp[:,:]
        E = mean_squared_loss(y, x)
        return E
    end
    

In [ ]:
forward_b(net, x, y, b, j) =
    let 
        tmp = randn(net.layer[j].n)
        tmp[:] = net.layer[j].b
        net.layer[j].b = b[:]
    
        for i=1:(size(net.layer)[1])
            x = net.layer[i].activation.(reshape(net.layer[i].W, net.layer[i].n, net.layer[i].m) * x .+ net.layer[i].b)
        end
         
        net.layer[j].b = tmp[:]
        E = mean_squared_loss(y, x)
        return E
    end

In [ ]:
backpropagation(net, x, y) =
    let
        for i=1:(size(net.layer)[1])
            net.layer[i].dW[:] = J(w-> forward_w(net,x, y, w, i), net.layer[i].dW[:])
            net.layer[i].db[:] = J(b-> forward_b(net,x, y, b, i), net.layer[i].db[:])
        end
    end


In [ ]:
update(net, x, y, α::Float64) =
    let
        for i=1:(size(net.layer)[1])
            net.layer[i].W -= α * net.layer[i].dW;
            net.layer[i].b -= α * net.layer[i].db;
        end
    end


In [ ]:
training(net, data_set, α::Float64) =
    let
        Loss_history = Float64[]
        for j = 1:5
            epoch_L = []
            for i = 1:(length(data_set))
                x = data_set[i][1]
                y = data_set[i][2]
                Ei = forward(net, x, y)
                push!(epoch_L, Ei)
                backpropagation(net, x, y)
                update(net, x, y, α)
            end
            push!(Loss_history, std(epoch_L))
        end

        return Loss_history
    end

In [ ]:
predict(net,x) = 
    let   
        for i=1:(size(net.layer)[1])
            x = net.layer[i].activation.(reshape(net.layer[i].W, net.layer[i].n, net.layer[i].m) * x .+ net.layer[i].b)
        end

        return argmax(x)
    end

In [ ]:
accuracy(network, data_set) =
    let
        return string("Accuracy: ", sum([predict(network,x[1]) == argmax(x[2]) ? 1 : 0 for x in data_set])/length(data_set)*100, "%")
    end

Utworzenie sieci

In [ ]:
# ReLU(x) = max(zero(x), x)
# σ(x) = one(x) / (one(x) + exp(-x))
# tanh(x) = 2.0 / (one(x) + exp(-2.0x)) - one(x)
# softmax(x)  =  exp.(x) ./ sum(exp.(x));
Random.seed!(2)
net = NeuralNetwork()
net.AddLayer(4, 4, ReLU)
net.AddLayer(4, 3, σ)
history = training(net, train, 0.01)
@show Plots.plot(history)

In [ ]:
predict(net,test[2][1])

In [ ]:
accuracy(net,test)